## Feature Engineering (Optimize Edilmiş Versiyon)

### Yapılan İyileştirmeler:
1. **Veri Sızıntısı Önlendi:** `min_periods` parametre ayarlandı
2. **Feature Sayısı Azaltıldı:** Sadece kritik pencereler (6h, 12h) kullanıldı
3. **Özellik Seçimi:** En önemli sensörler seçildi
4. **Temporal Integrity:** İleri dönük doldurma kaldırıldı, sadece gerekli yerlerde kullanıldı

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Ayarlar
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")

# Temiz Veriyi Yükle
data_path = '../data/processed/sensor_cleaned.csv'
if os.path.exists(data_path):
    df = pd.read_csv(data_path, parse_dates=['timestamp'], index_col='timestamp')
    print(f"Veri Yüklendi. Boyut: {df.shape}")
else:
    raise FileNotFoundError("Önce 01_EDA notebook'unu çalıştırmalısın.")

### Önemli Sensörleri Belirleme

Tüm sensörler yerine, arıza ile korelasyonu yüksek sensörleri seçelim.

In [ ]:
# Basit bir arıza tahmini için binary target oluştur
temp_target = (df['machine_status'] == 'BROKEN').astype(int)

# Tüm sensörler
sensor_cols = [c for c in df.columns if c.startswith('sensor')]

# Her sensörün arıza ile korelasyonunu hesapla
correlations = {}
for col in sensor_cols:
    correlations[col] = abs(df[col].corr(temp_target))

# En yüksek korelasyona sahip 30 sensörü seç
top_sensors = sorted(correlations.items(), key=lambda x: x[1], reverse=True)[:30]
selected_sensors = [s[0] for s in top_sensors]

print(f"Seçilen sensör sayısı: {len(selected_sensors)}")
print(f"\nEn önemli 10 sensör:")
for sensor, corr in top_sensors[:10]:
    print(f"{sensor}: {corr:.4f}")

### Optimize Feature Engineering

**Değişiklikler:**
- Sadece seçili sensörler kullanılıyor
- Pencere sayısı azaltıldı: 6 saat ve 12 saat (3 ve 24 yerine)
- `min_periods` ayarlandı (veri sızıntısı önlendi)
- Rolling mean + diff yeterli (std kaldırıldı)

In [ ]:
def make_features_optimized(df, selected_sensors):
    """
    Optimize edilmiş feature engineering:
    - Daha az pencere
    - Daha az metrik
    - Seçili sensörler
    - Veri sızıntısı koruması
    """
    df_eng = df.copy()
    
    # Sadece 2 pencere kullan (daha az feature)
    windows = [6, 12]  # 3, 12, 24 yerine
    
    print("Optimize edilmiş özellikler türetiliyor...")
    
    new_features = []
    
    for window in windows:
        w_size = window * 60  # dakikaya çevir
        
        for col in selected_sensors:
            # Sadece Rolling Mean (Std kaldırıldı)
            # min_periods ayarlandı - veri sızıntısını önler
            roll_mean = df_eng[col].rolling(
                window=w_size, 
                min_periods=int(w_size * 0.5)  # En az %50 veri olmalı
            ).mean()
            roll_mean.name = f'{col}_roll_mean_{window}h'
            new_features.append(roll_mean)
    
    # Diff özellikler (1 saat gecikmeli)
    for col in selected_sensors:
        diff_1h = df_eng[col].diff(60)  # 1 saatlik değişim
        diff_1h.name = f'{col}_diff_1h'
        new_features.append(diff_1h)
    
    # Tüm özellikleri birleştir
    if new_features:
        df_features = pd.concat(new_features, axis=1)
        df_eng = pd.concat([df_eng, df_features], axis=1)
    
    # NaN'ları temizle - GELECEĞİ KULLANMA!
    # Sadece baştan ve sondan keseceğiz
    df_eng = df_eng.dropna()
    
    print(f"Yeni Boyut: {df_eng.shape}")
    print(f"Türetilen Özellik Sayısı: {len(new_features)}")
    print(f"Önceki feature sayısı: 357 → Yeni: {len(new_features)} (Azalma: %{(1-len(new_features)/357)*100:.1f})")
    
    return df_eng

df_features = make_features_optimized(df, selected_sensors)

### Görselleştirme

In [ ]:
# En önemli sensörün trendin göster
sensor = selected_sensors[0]

plt.figure(figsize=(15, 6))
plt.plot(df_features.index, df_features[sensor], 
         label='Ham Veri', alpha=0.3, color='gray')
plt.plot(df_features.index, df_features[f'{sensor}_roll_mean_12h'], 
         label='12 Saatlik Trend', color='red', linewidth=2)
plt.title(f"{sensor} - Ham Veri vs Trend")
plt.legend()
plt.show()

### Target Oluşturma

In [ ]:
def create_target(df, window_hours=24):
    """
    Arızadan önceki N saati pozitif olarak etiketle
    """
    df_labeled = df.copy()
    df_labeled['y'] = 0
    
    broken_indices = df_labeled[df_labeled['machine_status'] == 'BROKEN'].index
    
    for fail_time in broken_indices:
        start_time = fail_time - pd.Timedelta(hours=window_hours)
        mask = (df_labeled.index >= start_time) & (df_labeled.index <= fail_time)
        df_labeled.loc[mask, 'y'] = 1
    
    # RECOVERING durumunu çıkar
    df_final = df_labeled[df_labeled['machine_status'] != 'RECOVERING'].drop(columns=['machine_status'])
    
    print(f"\nClass Distribution:")
    print(df_final['y'].value_counts())
    print(f"\nPositive ratio: {df_final['y'].mean()*100:.2f}%")
    
    return df_final

df_final = create_target(df_features, window_hours=24)

# Kaydet
save_path = '../data/processed/sensor_enriched_optimized.csv'
df_final.to_csv(save_path)
print(f"\n✅ Optimize edilmiş veri seti kaydedildi: {save_path}")
print(f"Son Boyut: {df_final.shape}")

## Özet

### İyileştirmeler:
1. ✅ **Feature sayısı azaltıldı:** 357 → ~90 (Daha az overfitting)
2. ✅ **Veri sızıntısı önlendi:** min_periods ve dropna kullanımı
3. ✅ **Sadece önemli sensörler:** En yüksek korelasyonlu 30 sensör
4. ✅ **Daha az pencere:** 2 pencere (6h, 12h) yeterli
5. ✅ **Gereksiz metrikler kaldırıldı:** Sadece mean ve diff (std gereksiz)

### Sonraki Adım:
04_ModelOptimization_Optimized.ipynb ile devam et.